# Small multiple pie charts for concept mentions per Author

In [3]:
import sys

!{sys.executable} -m pip install seaborn
!{sys.executable} -m pip install --force-reinstall git+https://github.com/climatepolicyradar/data-access@014f9bb9a9047e60d34e5d35d5725ef9b1ceb8a2


[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: python -m pip install --upgrade pip
  Cloning https://github.com/climatepolicyradar/data-access (to revision 014f9bb9a9047e60d34e5d35d5725ef9b1ceb8a2) to /private/var/folders/nt/2c78pgv94312v7_mmz24h6kc0000gn/T/pip-req-build-cup79qkw
  Running command git clone --filter=blob:none --quiet https://github.com/climatepolicyradar/data-access /private/var/folders/nt/2c78pgv94312v7_mmz24h6kc0000gn/T/pip-req-build-cup79qkw
  Running command git rev-parse -q --verify 'sha^014f9bb9a9047e60d34e5d35d5725ef9b1ceb8a2'
  Running command git fetch -q https://github.com/climatepolicyradar/data-access 014f9bb9a9047e60d34e5d35d5725ef9b1ceb8a2
  Resolved https://github.com/climatepolicyradar/data-access to commit 014f9bb9a9047e60d34e5d35d5725ef9b1ceb8a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [9]:
import sys

sys.path.append("../../..")
import os

from pathlib import Path
import itertools
from collections import Counter, defaultdict
from tqdm.auto import tqdm

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

from src.opensearch.index_data import get_dataset_and_filter_values

In [10]:
from cpr_data_access.models import GSTDocumentMetadata

GSTDocumentMetadata.schema()

{'title': 'GSTDocumentMetadata',
 'description': 'Metadata for a document in the Global Stocktake dataset.',
 'type': 'object',
 'properties': {'source': {'title': 'Source', 'type': 'string'},
  'author': {'title': 'Author', 'type': 'array', 'items': {'type': 'string'}},
  'validation_status': {'title': 'Validation Status',
   'enum': ['validated', 'not validated', 'error'],
   'type': 'string'},
  'themes': {'title': 'Themes', 'type': 'array', 'items': {'type': 'string'}},
  'types': {'title': 'Types', 'type': 'array', 'items': {'type': 'string'}},
  'version': {'title': 'Version', 'type': 'string'},
  'date': {'title': 'Date', 'type': 'string', 'format': 'date'},
  'link': {'title': 'Link', 'type': 'string'},
  'data_error_type': {'title': 'Data Error Type',
   'enum': ['source_incorrect',
    'outdated',
    'missing',
    'duplicate',
    'synthesis_error',
    'metadata_error',
    'incorrect_document'],
   'type': 'string'},
  'author_is_party': {'title': 'Author Is Party', 'type

In [11]:
cpr_dataset, _ = get_dataset_and_filter_values(
    os.environ["DOCS_DIR_GST"],
    Path(
        "/Users/kalyan/Documents/CPR/unfccc-global-stocktake-documents/CPR_UNFCCC_MASTER.csv"
    ),
    Path("../../../concepts").absolute(),
    limit=None,
)

INFO:src.opensearch.index_data:Loading scraper CSV
INFO:src.opensearch.index_data:Loading dataset of parsed documents


  0%|          | 0/1697 [00:00<?, ?it/s]

  0%|          | 0/1679 [00:00<?, ?it/s]

INFO:src.opensearch.index_data:Loaded 1679 documents. 0 documents failed to load.
INFO:src.opensearch.index_data:Loading spans from concepts directory
INFO:src.opensearch.index_data:Adding spans to dataset


0docs [00:00, ?docs/s]

In [12]:
author_span_types = defaultdict(list)

for document in tqdm(cpr_dataset.documents):
    if document.text_blocks is None:
        continue

    document_authors = document.document_metadata.author
    document_span_types = list()

    for block in document.text_blocks:
        if not block.spans:
            continue

        document_span_types.extend([span.type for span in block.spans])

    for author in document_authors:
        author_span_types[author].extend(document_span_types)

  0%|          | 0/1679 [00:00<?, ?it/s]

In [13]:
author_span_count = {k: dict(Counter(v)) for k, v in author_span_types.items()}
author_concept_count = {
    k: dict(Counter([i.split("–")[0].strip() for i in v]))
    for k, v in author_span_types.items()
}

In [14]:
author_is_party = dict()

for document in cpr_dataset.documents:
    if document.document_metadata.author not in author_is_party:
        author_is_party[
            document.document_metadata.author
        ] = document.document_metadata.author_is_party

TypeError: unhashable type: 'list'

In [35]:
results_df = pd.DataFrame(author_concept_count).fillna(0)
results_df = results_df.reindex(sorted(results_df.columns), axis=1)

display(results_df.head())

results_df.to_csv("concept-mention-count-by-author.csv")

,ANA Holdings INC (ANAHD),Aberystwyth University,Adaptation Research Alliance (ARA),Afghanistan,African Development Bank (AfDB),African Group,African Group Submission for the Second Technical Dialogue of the Global Stocktake,Air Transport Action Group (ATAG),Alaska Institute for Justice (AIJ),Albania,...,World Wide Fund for Nature (WWF),Wuppertal Institute on behalf of the NDC ASPECTS Project,Yemen,Youth Non-Governmental Organizations (YOUNGO),Zambia,Zimbabwe,and ESM2025,and World Resources Institute,solo Earth Observation (soloEO),the United Nations Climate Change High-Level Champions
Sectors,29.0,23.0,49.0,4444.0,276.0,11.0,32.0,26.0,22.0,5646.0,...,193.0,75.0,853.0,24.0,978,3362.0,50.0,1901.0,23.0,1901.0
Greenhouse Gases,23.0,2.0,2.0,3383.0,47.0,15.0,21.0,11.0,1.0,3492.0,...,78.0,32.0,800.0,3.0,910,2020.0,127.0,821.0,2.0,821.0
Barriers And Challenges,2.0,1.0,19.0,112.0,118.0,3.0,8.0,2.0,7.0,233.0,...,23.0,9.0,104.0,2.0,66,243.0,18.0,190.0,1.0,190.0
Deforestation,1.0,2.0,1.0,276.0,2.0,0.0,3.0,0.0,0.0,1082.0,...,40.0,6.0,97.0,0.0,228,670.0,0.0,449.0,2.0,449.0
Policy Instruments,8.0,4.0,69.0,1106.0,94.0,7.0,28.0,8.0,5.0,2375.0,...,66.0,38.0,335.0,34.0,499,1676.0,21.0,891.0,4.0,891.0


## temp reformatting test

In [5]:
df = pd.read_csv("concept-mention-count-by-author.csv", index_col=0)
df.head()

,ANA Holdings INC (ANAHD),Aberystwyth University,Adaptation Research Alliance (ARA),Afghanistan,African Development Bank (AfDB),African Group,African Group Submission for the Second Technical Dialogue of the Global Stocktake,Air Transport Action Group (ATAG),Alaska Institute for Justice (AIJ),Albania,...,World Wide Fund for Nature (WWF),Wuppertal Institute on behalf of the NDC ASPECTS Project,Yemen,Youth Non-Governmental Organizations (YOUNGO),Zambia,Zimbabwe,and ESM2025,and World Resources Institute,solo Earth Observation (soloEO),the United Nations Climate Change High-Level Champions
Sectors,29.0,23.0,49.0,4444.0,276.0,11.0,32.0,26.0,22.0,5646.0,...,193.0,75.0,853.0,24.0,978,3362.0,50.0,1901.0,23.0,1901.0
Greenhouse Gases,23.0,2.0,2.0,3383.0,47.0,15.0,21.0,11.0,1.0,3492.0,...,78.0,32.0,800.0,3.0,910,2020.0,127.0,821.0,2.0,821.0
Barriers And Challenges,2.0,1.0,19.0,112.0,118.0,3.0,8.0,2.0,7.0,233.0,...,23.0,9.0,104.0,2.0,66,243.0,18.0,190.0,1.0,190.0
Deforestation,1.0,2.0,1.0,276.0,2.0,0.0,3.0,0.0,0.0,1082.0,...,40.0,6.0,97.0,0.0,228,670.0,0.0,449.0,2.0,449.0
Policy Instruments,8.0,4.0,69.0,1106.0,94.0,7.0,28.0,8.0,5.0,2375.0,...,66.0,38.0,335.0,34.0,499,1676.0,21.0,891.0,4.0,891.0


In [8]:
df.stack().reset_index().to_csv("temp.csv")